<a href="https://colab.research.google.com/github/sidyr6002/product-sale-forecasting/blob/main/Sales_Forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [50]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [51]:
df = pd.read_csv('/content/drive/MyDrive/Scalar Projects/Sales Forecast/train.csv')

In [52]:
df

,ID,Store_id,Store_Type,Location_Type,Region_Code,Date,Holiday,Discount,#Order,Sales
0,T1000001,1,S1,L3,R1,2018-01-01,1,Yes,9,7011.84
1,T1000002,253,S4,L2,R1,2018-01-01,1,Yes,60,51789.12
2,T1000003,252,S3,L2,R1,2018-01-01,1,Yes,42,36868.20
3,T1000004,251,S2,L3,R1,2018-01-01,1,Yes,23,19715.16
4,T1000005,250,S2,L3,R4,2018-01-01,1,Yes,62,45614.52
...,...,...,...,...,...,...,...,...,...,...
188335,T1188336,149,S2,L3,R2,2019-05-31,1,Yes,51,37272.00
188336,T1188337,153,S4,L2,R1,2019-05-31,1,No,90,54572.64
188337,T1188338,154,S1,L3,R2,2019-05-31,1,No,56,31624.56
188338,T1188339,155,S3,L1,R2,2019-05-31,1,Yes,70,49162.41


In [53]:
df.rename(columns={'#Order': 'Order'}, inplace=True)

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188340 entries, 0 to 188339
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   ID             188340 non-null  object 
 1   Store_id       188340 non-null  int64  
 2   Store_Type     188340 non-null  object 
 3   Location_Type  188340 non-null  object 
 4   Region_Code    188340 non-null  object 
 5   Date           188340 non-null  object 
 6   Holiday        188340 non-null  int64  
 7   Discount       188340 non-null  object 
 8   Order          188340 non-null  int64  
 9   Sales          188340 non-null  float64
dtypes: float64(1), int64(3), object(6)
memory usage: 14.4+ MB


In [55]:
df.describe()

,Store_id,Holiday,Order,Sales
count,188340.000000,188340.000000,188340.000000,188340.000000
mean,183.000000,0.131783,68.205692,42784.327982
std,105.366308,0.338256,30.467415,18456.708302
min,1.000000,0.000000,0.000000,0.000000
25%,92.000000,0.000000,48.000000,30426.000000
50%,183.000000,0.000000,63.000000,39678.000000
75%,274.000000,0.000000,82.000000,51909.000000
max,365.000000,1.000000,371.000000,247215.000000


In [56]:
df.isna().sum()

,0
ID,0
Store_id,0
Store_Type,0
Location_Type,0
Region_Code,0
Date,0
Holiday,0
Discount,0
Order,0
Sales,0


There are no null values of handle

In [57]:
df.duplicated().sum()

np.int64(0)

# Proprocessing

In [58]:
df['Store_id'] = df['Store_id'].astype('category')
df['Region_Code'] = df['Region_Code'].astype('category')
df['Store_Type'] = df['Store_Type'].astype('category')
df['Location_Type'] = df['Location_Type'].astype('category')

In [59]:
df['Discount'] = df['Discount'].map({'Yes': 1, 'No': 0})

In [60]:
df['Date'] = pd.to_datetime(df['Date'])

In [61]:
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['Week_Of_Year'] = df['Date'].dt.isocalendar().week
df['Day_Of_Week'] = df['Date'].dt.dayofweek
df['Is_Weekend'] = df['Day_Of_Week'].apply(lambda x: 1 if x >= 5 else 0)
df['Is_Holiday_Discount'] = np.where((df['Holiday'] == 1) & (df['Discount'] == 1), 1, 0)

In [62]:
df

,ID,Store_id,Store_Type,Location_Type,Region_Code,Date,Holiday,Discount,Order,Sales,Year,Month,Day,Week_Of_Year,Day_Of_Week,Is_Weekend,Is_Holiday_Discount
0,T1000001,1,S1,L3,R1,2018-01-01,1,1,9,7011.84,2018,1,1,1,0,0,1
1,T1000002,253,S4,L2,R1,2018-01-01,1,1,60,51789.12,2018,1,1,1,0,0,1
2,T1000003,252,S3,L2,R1,2018-01-01,1,1,42,36868.20,2018,1,1,1,0,0,1
3,T1000004,251,S2,L3,R1,2018-01-01,1,1,23,19715.16,2018,1,1,1,0,0,1
4,T1000005,250,S2,L3,R4,2018-01-01,1,1,62,45614.52,2018,1,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188335,T1188336,149,S2,L3,R2,2019-05-31,1,1,51,37272.00,2019,5,31,22,4,0,1
188336,T1188337,153,S4,L2,R1,2019-05-31,1,0,90,54572.64,2019,5,31,22,4,0,0
188337,T1188338,154,S1,L3,R2,2019-05-31,1,0,56,31624.56,2019,5,31,22,4,0,0
188338,T1188339,155,S3,L1,R2,2019-05-31,1,1,70,49162.41,2019,5,31,22,4,0,1


In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188340 entries, 0 to 188339
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   ID                   188340 non-null  object        
 1   Store_id             188340 non-null  category      
 2   Store_Type           188340 non-null  category      
 3   Location_Type        188340 non-null  category      
 4   Region_Code          188340 non-null  category      
 5   Date                 188340 non-null  datetime64[ns]
 6   Holiday              188340 non-null  int64         
 7   Discount             188340 non-null  int64         
 8   Order                188340 non-null  int64         
 9   Sales                188340 non-null  float64       
 10  Year                 188340 non-null  int32         
 11  Month                188340 non-null  int32         
 12  Day                  188340 non-null  int32         
 13  Week_Of_Year  

In [64]:
df[['Store_Type']].value_counts()

,count
Store_Type,
S1,88752
S4,45924
S2,28896
S3,24768


In [65]:
df[['Location_Type']].value_counts()

,count
Location_Type,
L1,85140
L2,48504
L3,29928
L5,13932
L4,10836


In [66]:
df[['Region_Code']].value_counts()

,count
Region_Code,
R1,63984
R2,54180
R3,44376
R4,25800


In [67]:
df.groupby(['Store_Type', 'Location_Type', 'Region_Code']).size().reset_index(name='Count')

<ipython-input-67-814925091>:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(['Store_Type', 'Location_Type', 'Region_Code']).size().reset_index(name='Count')


,Store_Type,Location_Type,Region_Code,Count
0,S1,L1,R1,8772
1,S1,L1,R2,18060
2,S1,L1,R3,14964
3,S1,L1,R4,11352
4,S1,L2,R1,516
...,...,...,...,...
75,S4,L4,R4,0
76,S4,L5,R1,0
77,S4,L5,R2,0
78,S4,L5,R3,0


In [68]:
df_encoded = pd.get_dummies(df, columns=['Store_Type', 'Location_Type', 'Region_Code'], dtype=int)

In [69]:
df_encoded.sort_values(by=['Store_id', 'Date'], inplace=True)

In [70]:
df_encoded

,ID,Store_id,Date,Holiday,Discount,Order,Sales,Year,Month,Day,...,Store_Type_S4,Location_Type_L1,Location_Type_L2,Location_Type_L3,Location_Type_L4,Location_Type_L5,Region_Code_R1,Region_Code_R2,Region_Code_R3,Region_Code_R4
0,T1000001,1,2018-01-01,1,1,9,7011.84,2018,1,1,...,0,0,0,1,0,0,1,0,0,0
607,T1000608,1,2018-01-02,0,1,60,42369.00,2018,1,2,...,0,0,0,1,0,0,1,0,0,0
1046,T1001047,1,2018-01-03,0,1,72,50037.00,2018,1,3,...,0,0,0,1,0,0,1,0,0,0
1207,T1001208,1,2018-01-04,0,1,64,44397.00,2018,1,4,...,0,0,0,1,0,0,1,0,0,0
1752,T1001753,1,2018-01-05,0,1,63,47604.00,2018,1,5,...,0,0,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186722,T1186723,365,2019-05-27,0,1,55,39993.00,2019,5,27,...,0,1,0,0,0,0,0,1,0,0
186986,T1186987,365,2019-05-28,0,1,70,54132.00,2019,5,28,...,0,1,0,0,0,0,0,1,0,0
187479,T1187480,365,2019-05-29,0,0,31,22692.00,2019,5,29,...,0,1,0,0,0,0,0,1,0,0
187617,T1187618,365,2019-05-30,0,0,45,31671.00,2019,5,30,...,0,1,0,0,0,0,0,1,0,0


In [71]:
df_encoded['Sales_Lag7'] = df_encoded.groupby('Store_id')['Sales'].shift(7)
df_encoded['Sales_Lag30'] = df_encoded.groupby('Store_id')['Sales'].shift(30)

<ipython-input-71-83831484>:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_encoded['Sales_Lag7'] = df_encoded.groupby('Store_id')['Sales'].shift(7)
<ipython-input-71-83831484>:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_encoded['Sales_Lag30'] = df_encoded.groupby('Store_id')['Sales'].shift(30)


In [72]:
df_encoded['RollingStd_7'] = df_encoded.groupby('Store_id')['Sales'].rolling(window=7).std().reset_index(drop=True)

<ipython-input-72-1916958356>:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_encoded['RollingStd_7'] = df_encoded.groupby('Store_id')['Sales'].rolling(window=7).std().reset_index(drop=True)


In [73]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 188340 entries, 0 to 187983
Data columns (total 30 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   ID                   188340 non-null  object        
 1   Store_id             188340 non-null  category      
 2   Date                 188340 non-null  datetime64[ns]
 3   Holiday              188340 non-null  int64         
 4   Discount             188340 non-null  int64         
 5   Order                188340 non-null  int64         
 6   Sales                188340 non-null  float64       
 7   Year                 188340 non-null  int32         
 8   Month                188340 non-null  int32         
 9   Day                  188340 non-null  int32         
 10  Week_Of_Year         188340 non-null  UInt32        
 11  Day_Of_Week          188340 non-null  int32         
 12  Is_Weekend           188340 non-null  int64         
 13  Is_Holiday_Discount

In [74]:
df

,ID,Store_id,Store_Type,Location_Type,Region_Code,Date,Holiday,Discount,Order,Sales,Year,Month,Day,Week_Of_Year,Day_Of_Week,Is_Weekend,Is_Holiday_Discount
0,T1000001,1,S1,L3,R1,2018-01-01,1,1,9,7011.84,2018,1,1,1,0,0,1
1,T1000002,253,S4,L2,R1,2018-01-01,1,1,60,51789.12,2018,1,1,1,0,0,1
2,T1000003,252,S3,L2,R1,2018-01-01,1,1,42,36868.20,2018,1,1,1,0,0,1
3,T1000004,251,S2,L3,R1,2018-01-01,1,1,23,19715.16,2018,1,1,1,0,0,1
4,T1000005,250,S2,L3,R4,2018-01-01,1,1,62,45614.52,2018,1,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188335,T1188336,149,S2,L3,R2,2019-05-31,1,1,51,37272.00,2019,5,31,22,4,0,1
188336,T1188337,153,S4,L2,R1,2019-05-31,1,0,90,54572.64,2019,5,31,22,4,0,0
188337,T1188338,154,S1,L3,R2,2019-05-31,1,0,56,31624.56,2019,5,31,22,4,0,0
188338,T1188339,155,S3,L1,R2,2019-05-31,1,1,70,49162.41,2019,5,31,22,4,0,1


In [75]:
store_summary = df.groupby(['Store_id', 'Store_Type', 'Location_Type', 'Region_Code'], observed=True).agg(
    total_sales=('Sales', 'sum'),
    avg_sales=('Sales', 'mean'),
    total_orders=('Order', 'sum'),
    avg_order_count=('Order', 'mean'),
    num_days=('Date', 'count'),
    holiday_days=('Holiday', 'sum'),
    discount_days=('Discount', 'sum'),
    holiday_discount_days=('Is_Holiday_Discount', 'sum')
).reset_index()

In [76]:
df[df['Store_id'] == 1]

,ID,Store_id,Store_Type,Location_Type,Region_Code,Date,Holiday,Discount,Order,Sales,Year,Month,Day,Week_Of_Year,Day_Of_Week,Is_Weekend,Is_Holiday_Discount
0,T1000001,1,S1,L3,R1,2018-01-01,1,1,9,7011.84,2018,1,1,1,0,0,1
607,T1000608,1,S1,L3,R1,2018-01-02,0,1,60,42369.00,2018,1,2,1,1,0,0
1046,T1001047,1,S1,L3,R1,2018-01-03,0,1,72,50037.00,2018,1,3,1,2,0,0
1207,T1001208,1,S1,L3,R1,2018-01-04,0,1,64,44397.00,2018,1,4,1,3,0,0
1752,T1001753,1,S1,L3,R1,2018-01-05,0,1,63,47604.00,2018,1,5,1,4,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186569,T1186570,1,S1,L3,R1,2019-05-27,0,1,53,33075.00,2019,5,27,22,0,0,0
187165,T1187166,1,S1,L3,R1,2019-05-28,0,1,57,37317.00,2019,5,28,22,1,0,0
187391,T1187392,1,S1,L3,R1,2019-05-29,0,1,69,44652.00,2019,5,29,22,2,0,0
187962,T1187963,1,S1,L3,R1,2019-05-30,0,1,66,42387.00,2019,5,30,22,3,0,0


In [77]:
df.sort_values(['Store_id', 'Store_Type', 'Location_Type', 'Region_Code'])

,ID,Store_id,Store_Type,Location_Type,Region_Code,Date,Holiday,Discount,Order,Sales,Year,Month,Day,Week_Of_Year,Day_Of_Week,Is_Weekend,Is_Holiday_Discount
0,T1000001,1,S1,L3,R1,2018-01-01,1,1,9,7011.84,2018,1,1,1,0,0,1
607,T1000608,1,S1,L3,R1,2018-01-02,0,1,60,42369.00,2018,1,2,1,1,0,0
1046,T1001047,1,S1,L3,R1,2018-01-03,0,1,72,50037.00,2018,1,3,1,2,0,0
1207,T1001208,1,S1,L3,R1,2018-01-04,0,1,64,44397.00,2018,1,4,1,3,0,0
1752,T1001753,1,S1,L3,R1,2018-01-05,0,1,63,47604.00,2018,1,5,1,4,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186722,T1186723,365,S2,L1,R2,2019-05-27,0,1,55,39993.00,2019,5,27,22,0,0,0
186986,T1186987,365,S2,L1,R2,2019-05-28,0,1,70,54132.00,2019,5,28,22,1,0,0
187479,T1187480,365,S2,L1,R2,2019-05-29,0,0,31,22692.00,2019,5,29,22,2,0,0
187617,T1187618,365,S2,L1,R2,2019-05-30,0,0,45,31671.00,2019,5,30,22,3,0,0


In [78]:
df.sort_values(['Store_id', 'Store_Type', 'Location_Type', 'Region_Code']).groupby(['Store_id', 'Store_Type', 'Location_Type', 'Region_Code'], observed=True).agg(
     total_sales=('Sales', 'sum'),
     avg_sales=('Sales', 'mean'),
)

,,,,total_sales,avg_sales
Store_id,Store_Type,Location_Type,Region_Code,,
1,S1,L3,R1,17904396.24,34698.442326
2,S3,L1,R3,23908273.41,46333.863198
3,S4,L2,R1,32239395.66,62479.448953
4,S1,L1,R2,19856834.16,38482.236744
5,S1,L1,R3,22421598.81,43452.710872
...,...,...,...,...,...
361,S1,L3,R1,17297281.71,33521.863779
362,S1,L3,R3,17578014.99,34065.920523
363,S1,L1,R2,21340415.19,41357.393779


In [79]:
store_summary

,Store_id,Store_Type,Location_Type,Region_Code,total_sales,avg_sales,total_orders,avg_order_count,num_days,holiday_days,discount_days,holiday_discount_days
0,1,S1,L3,R1,17904396.24,34698.442326,30508,59.124031,516,68,236,34
1,2,S3,L1,R3,23908273.41,46333.863198,39318,76.197674,516,68,233,31
2,3,S4,L2,R1,32239395.66,62479.448953,59487,115.284884,516,68,235,35
3,4,S1,L1,R2,19856834.16,38482.236744,33477,64.877907,516,68,233,31
4,5,S1,L1,R3,22421598.81,43452.710872,35245,68.304264,516,68,235,35
...,...,...,...,...,...,...,...,...,...,...,...,...
360,361,S1,L3,R1,17297281.71,33521.863779,28714,55.647287,516,68,233,31
361,362,S1,L3,R3,17578014.99,34065.920523,29374,56.926357,516,68,235,35
362,363,S1,L1,R2,21340415.19,41357.393779,34898,67.631783,516,68,227,31
363,364,S2,L3,R2,15623047.92,30277.224651,23636,45.806202,516,68,226,33
